In [5]:
from dotenv import load_dotenv
import os
_ = load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# 1. 기본 LLM 체인 (Prompt + LLM)

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

llm.invoke("지구의 자전 주기는?")

AIMessage(content="지구의 자전 주기는 약 24시간입니다. 정확히는 23시간 56분 4초로, 이를 '태양일'이라고 하며, 우리가 일반적으로 하루라고 부르는 시간입니다. 이는 지구가 자전하면서 태양을 기준으로 한 위치가 돌아오는 시간을 의미합니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 15, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CzyHsRx6622eUMmY740CIcniohDFB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bd9ce-cbf6-7b21-8357-b7c4a0f33c37-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 68, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 프롬프트 템플릿 적용

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content="지구의 자전 주기는 약 24시간입니다. 정확히는 23시간 56분 4초로, 이를 '태양일'이라고 합니다. 이는 지구가 한 바퀴 자전하여 다시 태양을 같은 위치에서 볼 때까지 걸리는 시간을 기준으로 합니다. 그러나 우리는 일반적으로 하루를 24시간으로 계산합니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 29, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzyNDjQ2MsHcM7Rj8KOsmenC3FaW5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bd9d3-d61a-7ab3-9337-3009dad73d63-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 29, 'output_tokens': 77, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

### 출력 파서 적용

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 정확하게는 23시간 56분 4초(약 23.9345시간)로, 이 시간을 기준으로 지구는 한 번 자전하여 같은 점에서 다시 태양을 바라보게 됩니다. 일반적으로 우리는 하루를 24시간으로 나누어 사용합니다.'

# 2. 멀티 체인 (Multi-Chain)
LCEL(LangChain Expression Language)으로 파이프(`|`)와 `RunnableParallel`로 유연하게 조합

**1. 순차적 체인 (Sequential)**\
ex) 번역 -> 요약 -> 검토

`chain = A | B | C`

**2. 병렬 체인 (Parallel)**\
ex) 여러 관점에서 분석

`chain = RunnableParallel(a=A, b=B)`

**3. 조건부 분기 (Branching)**\
ex) 언어별 다른 프롬프트

`chain = RunnableBranch(...)`



### 기본 순차 체인

In [11]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = init_chat_model(model="gpt-4o-mini", api_key=api_key)

# 1단계: 한국어를 영어로 번역
translate_prompt = ChatPromptTemplate.from_template("다음 한국어를 영어로 번역하세요: {korean_word}")

# 2단계: 영어 단어 설명
explain_prompt = ChatPromptTemplate.from_template("다음 영어 단어를 한국어로 자세히 설명하세요: {english_word}")

# 체인 1: 번역
chain1 = translate_prompt | llm | StrOutputParser()

# 체인 2: 번역 결과를 입력으로 사용
chain2 = (
    {"english_word": chain1}
    | explain_prompt
    | llm
    | StrOutputParser()
)

# 실행
result = chain2.invoke({"korean_word": "인공지능"})
print(result)

"인공지능"은 "artificial intelligence"의 한국어 번역으로, 사람의 지능을 모방하거나 가상의 형태로 구현한 컴퓨터 시스템이나 프로그램을 의미합니다. 인공지능은 데이터를 분석하거나 학습하여 문제를 해결하고, 의사결정을 하며, 패턴을 인식하는 능력을 가집니다.

인공지능은 크게 두 가지로 나눌 수 있습니다. 첫 번째는 "약 인공지능(Weak AI)"로, 특정 작업이나 문제를 해결하기 위해 설계된 시스템입니다. 예를 들어, 음성 인식 시스템이나 이미지 인식 소프트웨어가 이에 해당합니다. 두 번째는 "강 인공지능(Strong AI)"으로, 이는 사람과 유사한 수준의 일반적인 지능을 가진 시스템을 의미하기도 하지만, 현재는 이론적인 개념에 더 가깝습니다.

또한, 인공지능은 머신러닝(기계 학습), 딥러닝(심층 학습), 자연어 처리 등이 포함된 다양한 기술과 분야를 포괄합니다. 이러한 기술들은 비즈니스, 의료, 자율주행차, 고객 서비스 등 여러 분야에서 혁신을 일으키고 있습니다. 인공지능의 발전은 인간의 삶을 더욱 편리하고 효율적으로 만들 수 있는 잠재력도 가지고 있지만, 윤리적 문제와 사회적 이슈도 함께 고려해야 하는 중요한 과제입니다.


### 다단계 순차 체인
`RunnablePassthrough`: 기존 데이터에 새로운 값을 추가하여 전달하는 역할

- `"topic": RunnablePassthrough()`
    - 입력: "기후 변화와 지속 가능한 발전"
    - 출력: {"topic" : "기후 변화와 지속 가능한 발전"}
- `RunnablePassthrough.assign(...)`
    - `assing` 내부 로직 실행
    - 결과: {"topic" : "...", "keywords" : "..."}

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = init_chat_model(model="gpt-4o-mini", api_key=api_key)

# 3단계 처리: 주제 분석 -> 개요 작성 -> 본문 작성
analyze_prompt = ChatPromptTemplate.from_template("다음 주제의 핵심 키워드 3개를 추출하세요: {topic}")

outline_prompt = ChatPromptTemplate.from_template(
    """다음 키워드를 바탕으로 글의 개요를 작성하세요:
    키워드: {keywords}
    원본 주제: {topic}"""
)

content_prompt = ChatPromptTemplate.from_template(
    """다음 개요를 바탕으로 300자 내외의 글을 작성하세요:
    개요: {outline}"""
)

# 체인 구성
chain = (
    {"topic": RunnablePassthrough()}
    | RunnablePassthrough.assign(
        keywords=analyze_prompt | llm | StrOutputParser()
        # keywords=(analyze_prompt | llm | StrOutputParser()) 도 가능
    )
    | RunnablePassthrough.assign(
        outline=outline_prompt | llm | StrOutputParser()
    )
    | content_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("기후 변화와 지속 가능한 발전")
print(result)

기후 변화는 지구의 평균 기온 상승과 기후 패턴의 변화를 일으키며, 이는 인류와 자연 생태계에 심각한 영향을 미치고 있습니다. 우리의 생존과 안전을 위협하는 이러한 변화는 지속 가능성과 환경 보호의 필요성을 더욱 강조합니다. 기후 변화의 주된 원인은 온실가스의 과도한 배출로, 이는 산업, 농업, 교통 등 인간 활동에서 비롯됩니다.

지속 가능한 발전은 경제적, 사회적 그리고 환경적 삼중적 균형을 이루어야 하며, 이는 미래 세대를 위한 필수 조건입니다. 환경 보호는 기후 변화에 대응하는 기본 원칙으로, 국제 사회는 파리 협정과 같은 다양한 이니셔티브를 통해 공동 대응을 모색하고 있습니다. 개인과 기업 또한 재생 가능 에너지 사용, 지속 가능한 농업 및 친환경 교통수단 도입 등으로 기여할 수 있습니다.

결론적으로, 기후 변화에 대한 긴급한 대응과 함께 지속 가능한 발전이 필요합니다. 이는 개인, 사회, 정부가 함께 힘을 모아 지구를 지키기 위한 행동을 취해야 함을 의미합니다. 우리의 미래는 우리의 손에 달려 있습니다.


### 병렬 체인
**RunnableParallel 기본 사용**

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = init_chat_model(model="gpt-4o-mini", api_key=api_key)

# 세 가지 관점에서 동시 분석
positive_prompt = ChatPromptTemplate.from_template("{topic}의 긍정적인 측면 3가지를 설명하세요.")

negative_prompt = ChatPromptTemplate.from_template("{topic}의 부정적인 측면 3가지를 설명하세요.")

neutral_prompt = ChatPromptTemplate.from_template("{topic}에 대한 객관적인 현황을 설명하세요.")

# 병렬 체인 구성
parallel_chain = RunnableParallel(
    positive = positive_prompt | llm | StrOutputParser(),
    negative= negative_prompt | llm | StrOutputParser(),
    neutral = neutral_prompt | llm | StrOutputParser(),
)

# 실행 (세 체인이 동시에 실행됨)
results = parallel_chain.invoke({"topic": {"원격 근무"}})

print("=== 긍정적 측면 ===")
print(results["positive"])
print("\n=== 부정적 측면 ===")
print(results["negative"])
print("\n=== 객관적 현황 ===")
print(results["neutral"])

=== 긍정적 측면 ===
원격 근무의 긍정적인 측면은 다음과 같습니다:

1. **유연한 근무 환경**: 원격 근무는 직원들이 자신에게 가장 잘 맞는 시간과 장소에서 작업할 수 있도록 합니다. 이로 인해 일과 삶의 균형이 개선되고, 개인의 라이프스타일에 맞춰 근무할 수 있는 자유가 생깁니다.

2. **시간 및 비용 절감**: 통근 시간과 비용이 줄어들어 직원들이 더 많은 시간을 업무에 집중하거나 개인적인 활동에 활용할 수 있습니다. 이는 스트레스를 줄이고, 전반적인 삶의 질을 향상시킬 수 있습니다.

3. **지리적 제약 최소화**: 원격 근무는 기업이 다양한 지역의 인재를 채용할 수 있게 해줍니다. 이를 통해 기업은 더 넓은 인재 풀에서 최고의 인재를 확보할 수 있으며, 직원들도 자신이 원하는 장소에서 경력을 쌓을 수 있는 기회를 가질 수 있습니다.

=== 부정적 측면 ===
원격 근무는 유연성과 편리함을 제공하지만, 몇 가지 부정적인 측면도 존재합니다. 다음은 그 중 3가지입니다:

1. **사회적 고립**: 원격 근무는 직원들이 동료와 직접 만나지 않고 일하는 환경을 조성합니다. 이로 인해 사회적 상호작용이 줄어들고, 결과적으로 외로움이나 고립감이 증가할 수 있습니다. 팀워크와 협업이 중요한 업무 환경에서는 이러한 고립이 팀의 사기와 생산성에 부정적인 영향을 미칠 수 있습니다.

2. **일과 삶의 경계 모호화**: 재택근무를 하게 되면 일하는 공간과 개인적인 공간이 겹치게 됩니다. 이로 인해 근무 시간이 늘어나거나, 휴식 없이 계속 일을 하게 되는 경우가 발생할 수 있습니다. 이러한 경계의 모호함은 스트레스를 증가시키고, 번아웃(탈진) 현상을 유발할 수 있습니다.

3. **의사소통의 어려움**: 비대면으로 의사소통을 하다 보면, 비언어적인 신호나 뉘앙스를 전달하기 어렵습니다. 이로 인해 오해가 발생하거나 정보 전달이 원활하지 않을 수 있습니다. 특히, 중요한 결정이나 복잡한 문제를 논의할 때 이러한 의사소통의 어려움은 업무의 효율성을 저하시킬 수

**병렬 결과 통합**

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = init_chat_model(model="gpt-4o-mini", api_key=api_key)

# 병렬 분석
analysis_chain = RunnableParallel(
    pros = ChatPromptTemplate.from_template("{topic}의 장점") | llm | StrOutputParser(),
    cons = ChatPromptTemplate.from_template("{topic}의 단점") | llm | StrOutputParser(),
)

# 결과 통합
synthesis_prompt = ChatPromptTemplate.from_template(
    """다음 분석을 종합하여 결론을 작성하세요:
    
    장점: {pros}
    단점: {cons}
    
    균형 잡힌 결론을 3문장으로 작성하세요."""
)

# 전체 체인: 병렬 분석 -> 통합
full_chain = (
    analysis_chain
    | synthesis_prompt
    | llm
    | StrOutputParser()
)

result = full_chain.invoke({"topic": "인공지능"})
print(result)

인공지능(AI)은 자동화, 정확성, 대량 데이터 처리 등 많은 장점을 제공하여 다양한 산업에서 혁신을 촉진하고 효율성을 높입니다. 그러나 편향성, 투명성 부족, 윤리적 문제와 같은 단점도 존재하며, 이는 AI의 활용을 더욱 신중하게 고려해야 할 필요성을 제기합니다. 따라서 AI의 잠재력을 최대한 활용하기 위해서는 기술의 발전과 함께 윤리적, 사회적 쟁점들을 해결해 나가는 것이 중요합니다.


### 조건부 분기
**RunnableBranch 사용**

In [17]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

llm = init_chat_model(model="gpt-4o-mini", api_key=api_key)

# 언어별 다른 프롬프트
korean_prompt = ChatPromptTemplate.from_template("다음 한국어 질문에 한국어로 답변하세요: {question}")
english_prompt = ChatPromptTemplate.from_template("Answer the following question in English: {question}")
default_prompt = ChatPromptTemplate.from_template("Please answer: {question}")

# 언어 감지 함수
def detect_language(input_dict):
    question = input_dict.get("question", "")
    # 간단한 한글 감지
    if any('\uac00' <= char <= '\ud7a3' for char in question):
        return "korean"
    return "english"

# 조건부 분기
branch_chain = RunnableBranch(
    # (조건 함수, 실행할 체인) 튜플 목록
    (lambda x: detect_language(x) == "korean", korean_prompt | llm | StrOutputParser()),
    (lambda x: detect_language(x) == "english", english_prompt | llm | StrOutputParser()),
    # 기본값 (조건에 맞지 않을 때)
    default_prompt | llm | StrOutputParser(),
)

# 테스트
result_kr = branch_chain.invoke({"question": "파이썬이란 무엇인가요?"})
result_en = branch_chain.invoke({"question": "What is Python?"})

print("한국어 질문 결과:", result_kr)
print("영어 질문 결과:", result_en)

한국어 질문 결과: 파이썬(Python)은 고급 프로그래밍 언어로, 1991년 귀도 반 로섬(Guido van Rossum)이 처음 개발했습니다. 이 언어는 코드를 간결하고 쉽게 읽을 수 있도록 설계되어 있어, 프로그래밍 초보자뿐만 아니라 전문가들 사이에서도 널리 사용됩니다. 파이썬은 다양한 분야에서 활용되며, 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립트 작성 등 여러 용도로 쓰입니다. 또한, 풍부한 라이브러리와 프레임워크를 제공하여 개발자들이 효율적으로 작업할 수 있도록 돕고 있습니다.
영어 질문 결과: Python is a high-level, interpreted programming language known for its clear syntax and readability. It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It is widely used for a variety of applications, such as web development, data analysis, artificial intelligence, scientific computing, automation, and more. Python's extensive standard library and vibrant community contribute to its popularity, making it a go-to choice for both beginners and experienced developers.


**함수 기반 라우팅**

In [19]:
from langchain_core.runnables import RunnableLambda

def route_by_topic(input_dict):
    """주제에 따라 다른 체인 선택"""
    topic = input_dict.get("topic", "").lower()

    if "code" in topic or "programming" in topic:
        return "technical"
    elif "business" in topic or "strategy" in topic:
        return "business"
    else:
        return "general"
    
# 주제별 체인 정의
chains = {
    "technical": ChatPromptTemplate.from_template("기술 전문가로서 답변: {question}")
    | llm
    | StrOutputParser(),

    "business": ChatPromptTemplate.from_template("비즈니스 컨설턴트로서 답변: {question}") 
    | llm 
    | StrOutputParser(),

    "general": ChatPromptTemplate.from_template("일반적인 관점에서 답변: {question}") 
    | llm 
    | StrOutputParser(),
}

# 라우팅 체인
def routing_chain(input_dict):
    route = route_by_topic(input_dict)
    return chains[route].invoke(input_dict)

router = RunnableLambda(routing_chain)

# 테스트
result = router.invoke({
    "topic": "Python programming",
    "question": "효율적인 코드 작성 방법은?"
})

print(result)

효율적인 코드를 작성하는 방법은 여러 가지가 있으며, 다음과 같은 몇 가지 원칙과 기법을 고려할 수 있습니다.

1. **명확한 요구사항 정의**: 코드를 작성하기 전에 요구사항을 명확히 이해하고 정의하는 것이 중요합니다. 이를 통해 불필요한 로직을 피하고 더 나은 설계를 할 수 있습니다.

2. **모듈화**: 코드를 작고 재사용 가능한 모듈로 나누는 것이 좋습니다. 함수나 클래스 등으로 나누어 특정 기능을 담당하게 하여 코드의 가독성과 유지보수성을 높입니다.

3. **주석 및 문서화**: 복잡한 코드에는 적절한 주석을 추가하고, 전체 코드를 문서화하여 설명하는 것이 중요합니다. 이는 다른 개발자뿐만 아니라 미래의 자신에게도 도움이 됩니다.

4. **코드 리뷰**: 동료에게 코드를 리뷰 받는 것은 품질을 높이는 좋은 방법입니다. 다른 시각에서의 피드백을 통해 버그를 발견하거나 개선할 수 있는 점을 찾을 수 있습니다.

5. **효율적인 알고리즘 및 자료 구조 선택**: 문제 해결에 적합한 알고리즘과 자료 구조를 선택하는 것은 코드의 성능에 큰 영향을 미칩니다. 시간 복잡도와 공간 복잡도를 고려하여 최적의 선택을 하세요.

6. **테스트 주도 개발(TDD)**: 개발 초기 단계에서부터 테스트 코드를 작성하여 예상치 못한 버그를 예방할 수 있습니다. 이를 통해 코드 변경 시에도 기존 기능이 정상적으로 작동하는지 확인할 수 있습니다.

7. **코드 스타일 일관성**: 코드 스타일 가이드(예: PEP8 for Python)를 따르며 일관성을 유지하는 것이 중요합니다. 이를 통해 팀 내에서 협업할 때 가독성이 향상됩니다.

8. **성능 프로파일링**: 코드가 복잡해지거나 느려질 경우, 성능을 프로파일링하여 병목현상을 찾아 최적화하는 것이 필요합니다. 이를 통해 실제로 어떤 부분이 성능 저하의 원인인지 파악할 수 있습니다.

9. **적절한 라이브러리 및 프레임워크 사용**: 필요한 기능을 직접 구현하기보다는 검증된 라이브러리나 프레임워크를 활용하는 것이 좋습

### 복합 패턴
**RAG 스타일 멀티 체인**

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

llm = init_chat_model(model="gpt-4o-mini", api_key=api_key)

# 가상의 검색 함수
def retrieve_context(query: str) -> str:
    # 실제로는 벡터스토어 검색
    return f"검색된 컨텍스트: {query}에 대한 정보..."

# RAG 체인 구성
rag_chain = (
    # 1. 쿼리와 컨텍스트 병렬 준비
    RunnableParallel(
        question = RunnablePassthrough(),
        context = RunnableLambda(lambda x: retrieve_context(x["question"]))
    )
    # 2. 프롬프트 생성
    | ChatPromptTemplate.from_template(
        """컨텍스트를 참고하여 질문에 답변하세요.
        
        컨텍스트: {context}
        질문: {question}
        답변: """
    )
    # 3. LLM 호출
    | llm
    # 4. 출력 파싱
    | StrOutputParser()
)

result = rag_chain.invoke({"question": "LangChain이란?"})
print(result)

LangChain은 자연어 처리(NLP)와 인공지능(AI) 모델을 활용하여 다양한 언어 기반 애플리케이션을 개발하는 데 도움을 주는 프레임워크입니다. 이 프레임워크는 대화형 AI 시스템, 정보 검색, 텍스트 생성 및 변환, 데이터 분석 등 여러 분야에서 사용할 수 있는 유용한 도구와 기능을 제공합니다. LangChain은 다양한 모델과 API를 통합하여 개발자들이 효율적으로 언어 관련 솔루션을 구축할 수 있도록 지원합니다.


### RunnablePassthrough 활용
**입력 데이터 보존**

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# 원본 입력을 유지하면서 추가 데이터 할당
chain = (
    RunnablePassthrough.assign(
        # summary 키에 요약 결과 추가
        summary=ChatPromptTemplate.from_template("{text}를 요약") | llm | StrOutputParser()
    )
    |
    RunnablePassthrough.assign(
        # keywords 키에 키워드 추가
        keywords=ChatPromptTemplate.from_template("{text}의 키워드") | llm | StrOutputParser()
    )
)

result = chain.invoke({"text": "긴 문서 내용..."})
# result = {"text": "긴 문서 내용...", "summary": "...", "keywords": "..."}

### RunnableLambda 활용
**데이터 변환**

In [ ]:
from langchain_core.runnables import RunnableLambda

# 입력 변환 함수
def preprocess(input_dict):
    return {
        "processed_text": input_dict["text"].strip().lower(),
        "original": input_dict["text"]
    }

# 출력 변환 함수
def postprocess(output):
    return {
        "result": output,
        "length": len(output)
    }

chain = (
    RunnableLambda(preprocess)
    | ChatPromptTemplate.from_template("분석: {processed_text}")
    | llm
    | StrOutputParser()
    | RunnableLambda(postprocess)
)

# 요약

| 패턴 | 구현 방법 | 사용 시점 |
| :--- | :--- | :--- |
| **순차적 체인** | `chain1 \| chain2` | 단계별 처리 |
| **병렬 체인** | `RunnableParallel(a=..., b=...)` | 독립 작업 동시 실행 |
| **조건부 분기** | `RunnableBranch(...)` | 입력 기반 라우팅 |
| **데이터 보존** | `RunnablePassthrough.assign()` | 중간 결과 축적 |
| **데이터 변환** | `RunnableLambda(func)` | 전처리/후처리 |

### 멀티 체인 설계 팁
- 병렬 실행: API 호출이 많으면 `RunnableParallel`로 시간 단축
- 중간 결과 보존: `assign()`으로 디버깅과 추적 용이
- 에러 처리: 각 체인에 `with_fallbacks()` 추가 고려
- 복잡한 로직: 너무 복잡하면 여러 체인으로 분리